In [10]:
import xml.etree.ElementTree as ET
from collections import deque

In [81]:
def getRoot(filename: str):
    # in original xml file, we didn't add a root tag for further content appeding, so root tag is added when processing xml.
    with open(filename, "r") as f:
        filestr = f.read()
    filestr = "<contract>" + filestr + "</contract>"
    with open("p_"+filename, "w") as f:
        f.write(filestr)
    
    
    tree = ET.parse("p_"+filename)
    root = tree.getroot()
    return root

In [87]:
def node_equal(node1, node2):
    key = ["caller", "callee",  "gas", "input","value", "remainGas"] # add "input","value", "remainGas"
    for k in key:
        if node1[k] != node2[k]:
            return False
    if len(node1["opcodes"]) != len(node2["opcodes"]):
        return False
    for i in range(len(node1["opcodes"])):
        if node1["opcodes"][i].tag != node2["opcodes"][i].tag:
            return False
    return True

class Oracle:
    __hit = False
    def judge(self, call_tree):
        pass
    
    def printOracle(self):
        pass
    
class Timestamp(Oracle):
    def judge(self, call_tree):
        for item in call_tree.root["opcodes"]:
            if item.tag == "timestamp":
                return True
        return False
                
    def printOracle(self):
        print("ChainHacker: Detected a timestamp vulnerability in fuzzing process. Please check if you used timestamp command in your contract.")
    
class Reenterency(Oracle):
    def judge(self, call_tree):
        root = call_tree.getRoot()
        
        # make tree into a list
        def tolist(node):
            ret = [node]
            for child in node["child"]:
                ret += tolist(child)
            return ret
        tree_list = tolist(root)
        
        for i in range(1, len(tree_list)-1):
            icall = tree_list[i]
            if node_equal(icall, root):
                return True
        return False
    
    def printOracle(self):
        print("ChainHacker: Detected a reentrency vulnerability! Please check if you send ether before confirming balance.")
        
class Overflow(Oracle):
    def judge(self, call_tree):
        for item in call_tree.root["opcodes"]:
            if item.tag == "add" and item.text == "overflow":
                return True
        return False
    
    def printOracle(self):
        print("ChainHacker: Detected overflow vulnerability! Please check if you use safed arithmetic methods.")
        
class GaslessSend(Oracle):
    def judge(self, call_tree):
        root = call_tree.getRoot()
        call_value = int(root["value"])
        call_gas = int(root["gas"])
        call_input = root["input"]
        call_error = root["error"]
        
        # why len(call_input) == 2? that means "[]", i.e. real input length == 0
        if call_value > 0 and call_gas == 2300 and len(call_input) == 2 and call_error == "out of gas":
            return True
        return False
    
    def printOracle(self):
        print("ChainHacker: Detected gasless sending vulnerability! Send operation may cause failure due to high cost fallback function. Please use call function instead.")
        
    

    
def init_oracles():
    return (Timestamp(), Reenterency(), Overflow(), GaslessSend())

In [85]:
class Calltree:
    def __init__(self, rootcall):
        def makeNode(call):
            assert call.tag == "call"
            
            ret = {}
            for tag in call:
                ret[tag.tag] = tag if tag.tag == "opcodes" else tag.text
            ret["child"] = []
            for opcode in ret["opcodes"]:
                if opcode.tag == "call":
                    ret["child"].append(makeNode(opcode))
            return ret
        self.root = makeNode(rootcall)
    
    def print_tree(self):
        def _pt(callmsg, depth):
            indent = " "*depth
            print(indent+"caller:"+callmsg["caller"])
            print(indent+"callee:"+callmsg["callee"])
            
            for child in callmsg["child"]:
                _pt(child, depth+1)
            
        _pt(self.root, 0)
        
    def getRoot(self):
        return self.root

if __name__ == "__main__":
    #init
    fuzzerOutput = getRoot("test.xml")
    oracles = init_oracles()
    
    for root_tag in fuzzerOutput:
        if root_tag.tag == "call":
            # making tree
            call_tree = Calltree(root_tag)

            # test oracle
            for oracle in oracles:
                if oracle.judge(call_tree):
                    oracle.printOracle()
        
        
        
    
        

ChainHacker: Detected overflow vulnerability! Please check if you use safed arithmetic methods.
